In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    accuracy_score,
    auc,
    average_precision_score,
    classification_report,
    confusion_matrix,
    precision_recall_curve,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import (
    cross_val_predict,
    cross_val_score,
    train_test_split,
    KFold,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, RobustScaler

In [13]:
project_root = "D:\\Projects\\Summer-2024-ECE-597-Group8"
data_path = os.path.join(project_root, "data", "processed", "features_tfidf_labels.csv")
model_path = os.path.join(project_root, 'data', 'models', 'best_cnn_model.keras')
random_state = 42
test_size = 0.1

In [3]:
df = pd.read_csv(data_path)
df = df.drop(columns=["js_code"])

In [4]:
def log_transform(x):
    return np.log1p(x)

In [5]:
pipeline_log_transform_RobustScaler = Pipeline(
    steps=[
        ("log_transform", FunctionTransformer(log_transform)),
        ("RobustScaler", RobustScaler()),
    ]
)

features_preprocessor = ColumnTransformer(
    transformers=[
        (
            "log_transform_RobustScaler",
            pipeline_log_transform_RobustScaler,
            [
                "Word_Count",
                "Homoglyphs",
                "Total_Abnormal_Count",
                "html_tags",
            ],
        ),
    ],
    remainder="passthrough",
)

In [6]:
X = features_preprocessor.fit_transform(df.drop(columns=["Label"]))
y = df["Label"].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=random_state
)

In [7]:
y_train

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import optuna
from tensorflow.keras import backend as K

# Reshape the data for CNN
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

# Define a custom F1-score metric
def f1_score_metric(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.round(K.cast(y_pred, 'float32'))
    tp = K.sum(K.cast(y_true * y_pred, 'float32'), axis=0)
    tn = K.sum(K.cast((1 - y_true) * (1 - y_pred), 'float32'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float32'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float32'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())
    
    f1 = 2 * p * r / (p + r + K.epsilon())
    f1 = K.mean(f1)
    return f1

# Define the model creation function
def create_model(trial):
    model = Sequential()
    model.add(Input(shape=(X_train_reshaped.shape[1], 1)))
    model.add(Conv1D(filters=trial.suggest_int('filters', 16, 128), 
                     kernel_size=trial.suggest_int('kernel_size', 2, 5), 
                     activation='relu'))
    model.add(MaxPooling1D(pool_size=trial.suggest_int('pool_size', 2, 4)))
    model.add(Conv1D(filters=trial.suggest_int('filters2', 16, 128), 
                     kernel_size=trial.suggest_int('kernel_size2', 2, 5), 
                     activation='relu'))
    model.add(MaxPooling1D(pool_size=trial.suggest_int('pool_size2', 2, 4)))
    model.add(Flatten())
    model.add(Dense(units=trial.suggest_int('units', 32, 256), activation='relu'))
    model.add(Dropout(rate=trial.suggest_float('dropout_rate', 0.2, 0.5)))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=trial.suggest_categorical('optimizer', ['adam', 'rmsprop']),
                  loss='binary_crossentropy',
                  metrics=[f1_score_metric])
    return model

# Define the objective function for Optuna
def objective(trial):
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    f1_scores = []

    for train_index, valid_index in skf.split(X_train_reshaped, y_train):
        X_t, X_v = X_train_reshaped[train_index], X_train_reshaped[valid_index]
        y_t, y_v = y_train[train_index], y_train[valid_index]

        model = create_model(trial)
        early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
        model.fit(X_t, y_t, 
                  epochs=trial.suggest_int('epochs', 10, 50), 
                  batch_size=trial.suggest_int('batch_size', 32, 256),
                  validation_data=(X_v, y_v), 
                  callbacks=[early_stopping], 
                  verbose=0)
        
        y_pred = (model.predict(X_v) > 0.5).astype("int32")
        f1 = f1_score(y_v, y_pred, pos_label=1)
        f1_scores.append(f1)
    
    return np.mean(f1_scores)

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=6)

# Print the best parameters
print('Best trial:')
trial = study.best_trial
print(f'Value: {trial.value}')
print('Params:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

[I 2024-07-02 04:27:10,125] A new study created in memory with name: no-name-39b46be8-e5c8-4b54-90c2-de9040835dfb


266/266 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step
266/266 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step
266/266 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step
266/266 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step
266/266 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step
266/266 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step
266/266 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step
266/266 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step
266/266 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step
266/266 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step
266/266 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step
266/266 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step


[I 2024-07-02 04:32:49,599] Trial 5 finished with value: 0.8115591237029692 and parameters: {'filters': 49, 'kernel_size': 4, 'pool_size': 3, 'filters2': 64, 'kernel_size2': 2, 'pool_size2': 3, 'units': 200, 'dropout_rate': 0.45353720280360194, 'optimizer': 'rmsprop', 'epochs': 15, 'batch_size': 119}. Best is trial 5 with value: 0.8115591237029692.


266/266 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step
266/266 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step


[I 2024-07-02 04:33:53,542] Trial 3 finished with value: 0.8012086759780738 and parameters: {'filters': 61, 'kernel_size': 4, 'pool_size': 4, 'filters2': 69, 'kernel_size2': 5, 'pool_size2': 4, 'units': 88, 'dropout_rate': 0.4732658279711917, 'optimizer': 'adam', 'epochs': 33, 'batch_size': 222}. Best is trial 5 with value: 0.8115591237029692.


...

266/266 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step


[I 2024-07-02 06:16:42,671] Trial 48 finished with value: 0.8489754289359647 and parameters: {'filters': 93, 'kernel_size': 4, 'pool_size': 2, 'filters2': 62, 'kernel_size2': 3, 'pool_size2': 2, 'units': 208, 'dropout_rate': 0.4981406534873368, 'optimizer': 'adam', 'epochs': 32, 'batch_size': 214}. Best is trial 27 with value: 0.8618945791353747.


266/266 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


[I 2024-07-02 06:16:44,516] Trial 49 finished with value: 0.8545192064009268 and parameters: {'filters': 93, 'kernel_size': 5, 'pool_size': 2, 'filters2': 58, 'kernel_size2': 2, 'pool_size2': 2, 'units': 237, 'dropout_rate': 0.3674243746928667, 'optimizer': 'adam', 'epochs': 45, 'batch_size': 194}. Best is trial 27 with value: 0.8618945791353747.


Best trial:
Value: 0.8618945791353747
Params:
    filters: 105
    kernel_size: 4
    pool_size: 2
    filters2: 79
    kernel_size2: 3
    pool_size2: 2
    units: 219
    dropout_rate: 0.28897492290973636
    optimizer: adam
    epochs: 42
    batch_size: 239


In [14]:
# Ensure the directory exists
os.makedirs(os.path.dirname(model_path), exist_ok=True)

# Create the model with the best parameters from Optuna
best_params = study.best_trial.params

model = Sequential()
model.add(Input(shape=(X_train_reshaped.shape[1], 1)))
model.add(Conv1D(filters=best_params['filters'], 
                 kernel_size=best_params['kernel_size'], 
                 activation='relu'))
model.add(MaxPooling1D(pool_size=best_params['pool_size']))
model.add(Conv1D(filters=best_params['filters2'], 
                 kernel_size=best_params['kernel_size2'], 
                 activation='relu'))
model.add(MaxPooling1D(pool_size=best_params['pool_size2']))
model.add(Flatten())
model.add(Dense(units=best_params['units'], activation='relu'))
model.add(Dropout(rate=best_params['dropout_rate']))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=best_params['optimizer'],
              loss='binary_crossentropy',
              metrics=[f1_score_metric])

# Train the model with the entire training data
history = model.fit(X_train_reshaped, y_train, 
                    epochs=best_params['epochs'], 
                    batch_size=best_params['batch_size'], 
                    validation_split=0.2)

# Save the trained model to a file
model.save(model_path)
print(f"Model saved to {model_path}")

Epoch 1/42
86/86 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - f1_score_metric: 0.0060 - loss: 0.3420 - val_f1_score_metric: 7.2905e-05 - val_loss: 0.1772
Epoch 2/42
86/86 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - f1_score_metric: 8.9766e-04 - loss: 0.1694 - val_f1_score_metric: 0.0054 - val_loss: 0.1363
Epoch 3/42
86/86 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - f1_score_metric: 0.0077 - loss: 0.1407 - val_f1_score_metric: 0.0042 - val_loss: 0.1340
Epoch 4/42
86/86 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - f1_score_metric: 0.0096 - loss: 0.1257 - val_f1_score_metric: 0.0131 - val_loss: 0.1033
Epoch 5/42
86/86 ━━━━━━━━━━━━━━━━━━━━ 5s 53ms/step - f1_score_metric: 0.0117 - loss: 0.1070 - val_f1_score_metric: 0.0139 - val_loss: 0.0982
Epoch 6/42
86/86 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - f1_score_metric: 0.0136 - loss: 0.1028 - val_f1_score_metric: 0.0144 - val_loss: 0.0862
Epoch 7/42
86/86 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - f1_score_metric: 0.0142 - loss: 0.0882 - val_f1_score_metric: 0.0122 - val_loss: 0.0899
Epoch

In [15]:
# Reshape the test data for CNN
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Load the saved model
model = tf.keras.models.load_model(model_path, custom_objects={'f1_score_metric': f1_score_metric})
print(f"Model loaded from {model_path}")

# Make predictions on the test data
y_pred_probs = model.predict(X_test_reshaped)
y_pred = (y_pred_probs > 0.5).astype(int)

# Generate the classification report
report = classification_report(y_test, y_pred, target_names=['Class 0', 'Class 1'])
print("Classification Report:\n")
print(report)

Model loaded from D:\Projects\Summer-2024-ECE-597-Group8\data\models\best_cnn_model.keras
89/89 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Classification Report:

              precision    recall  f1-score   support

     Class 0       0.99      0.99      0.99      2588
     Class 1       0.90      0.86      0.88       241

    accuracy                           0.98      2829
   macro avg       0.94      0.93      0.93      2829
weighted avg       0.98      0.98      0.98      2829

